In [29]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [30]:
client = MongoClient("mongo")
database = client['projet']
collection = database['pokemon']

In [229]:
df_pok = pd.read_csv("./data/pokemon.csv")
df_pok.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp Atk,Sp Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [230]:
df_pok.fillna('', inplace=True)

#### Nettoyage des données

In [231]:
collection.drop()

#### Import des données

In [232]:
l_d = df_pok.to_dict(orient='records')
collection.insert_many(l_d)

#### Requête 1
Recherche des pokémons qui sont de type "feu" et qui sont de 3ème génération

In [84]:
cur = collection.find({"$and": [{"$or": [{"Type 1": "Fire"}, {"Type 2": "Fire"}]}, {"Generation": 3}]})
list(cur)

[{'_id': ObjectId('602fe9123641630013fcaf7c'),
  '#': 255,
  'Name': 'Torchic',
  'Type 1': 'Fire',
  'Type 2': '',
  'Total': 310,
  'HP': 45,
  'Attack': 60,
  'Defense': 40,
  'Sp Atk': 70,
  'Sp Def': 50,
  'Speed': 45,
  'Generation': 3,
  'Legendary': False},
 {'_id': ObjectId('602fe9123641630013fcaf7d'),
  '#': 256,
  'Name': 'Combusken',
  'Type 1': 'Fire',
  'Type 2': 'Fighting',
  'Total': 405,
  'HP': 60,
  'Attack': 85,
  'Defense': 60,
  'Sp Atk': 85,
  'Sp Def': 60,
  'Speed': 55,
  'Generation': 3,
  'Legendary': False},
 {'_id': ObjectId('602fe9123641630013fcaf7e'),
  '#': 257,
  'Name': 'Blaziken',
  'Type 1': 'Fire',
  'Type 2': 'Fighting',
  'Total': 530,
  'HP': 80,
  'Attack': 120,
  'Defense': 70,
  'Sp Atk': 110,
  'Sp Def': 70,
  'Speed': 80,
  'Generation': 3,
  'Legendary': False},
 {'_id': ObjectId('602fe9123641630013fcaf7f'),
  '#': 257,
  'Name': 'BlazikenMega Blaziken',
  'Type 1': 'Fire',
  'Type 2': 'Fighting',
  'Total': 630,
  'HP': 80,
  'Attack': 160

#### Requête 2
Recherche des pokémons qui sont légendaires, qui ont au moins 120 points d'attaque et qui sont de 4ème génération

In [226]:
cur = collection.find({"$and": [{"Legendary": True}, {"Attack": {"$gte": 120 }}, {"Generation": 4}]})
list(cur)

[]

#### Requête 3
Recherche des 5 pokémons qui sont légendaires, qui ont moins de 100 points d'attaque, et qui ont la meilleure défense

In [233]:
cur = collection.find({
    "$and": [{"Legendary": True}, {"Attack": {"$lt": 120 }}, {"Generation": 4}]
}).sort([("Defense", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('602ffe143641630013fcb3a1'),
  '#': 480,
  'Name': 'Uxie',
  'Type 1': 'Psychic',
  'Type 2': '',
  'Total': 580,
  'HP': 75,
  'Attack': 75,
  'Defense': 130,
  'Sp Atk': 75,
  'Sp Def': 130,
  'Speed': 95,
  'Generation': 4,
  'Legendary': True},
 {'_id': ObjectId('602ffe143641630013fcb3a8'),
  '#': 487,
  'Name': 'GiratinaAltered Forme',
  'Type 1': 'Ghost',
  'Type 2': 'Dragon',
  'Total': 680,
  'HP': 150,
  'Attack': 100,
  'Defense': 120,
  'Sp Atk': 100,
  'Sp Def': 120,
  'Speed': 90,
  'Generation': 4,
  'Legendary': True},
 {'_id': ObjectId('602ffe143641630013fcb3a6'),
  '#': 485,
  'Name': 'Heatran',
  'Type 1': 'Fire',
  'Type 2': 'Steel',
  'Total': 600,
  'HP': 91,
  'Attack': 90,
  'Defense': 106,
  'Sp Atk': 130,
  'Sp Def': 106,
  'Speed': 77,
  'Generation': 4,
  'Legendary': True},
 {'_id': ObjectId('602ffe143641630013fcb3a2'),
  '#': 481,
  'Name': 'Mesprit',
  'Type 1': 'Psychic',
  'Type 2': '',
  'Total': 580,
  'HP': 80,
  'Attack': 105,
  'De

#### Requête 4
Compter le nombre de pokémons pour chaque type principal (Type 1)

In [106]:
cur = collection.aggregate([{"$group" : {"_id" : {"Type": "$Type 1"}, "pokemonsByType": {"$sum": 1}}}])
list(cur)

[{'_id': {'Type': 'Grass'}, 'pokemonsByType': 70},
 {'_id': {'Type': 'Dark'}, 'pokemonsByType': 31},
 {'_id': {'Type': 'Poison'}, 'pokemonsByType': 28},
 {'_id': {'Type': 'Fighting'}, 'pokemonsByType': 27},
 {'_id': {'Type': 'Dragon'}, 'pokemonsByType': 32},
 {'_id': {'Type': 'Fairy'}, 'pokemonsByType': 17},
 {'_id': {'Type': 'Bug'}, 'pokemonsByType': 69},
 {'_id': {'Type': 'Normal'}, 'pokemonsByType': 98},
 {'_id': {'Type': 'Psychic'}, 'pokemonsByType': 57},
 {'_id': {'Type': 'Steel'}, 'pokemonsByType': 27},
 {'_id': {'Type': 'Flying'}, 'pokemonsByType': 4},
 {'_id': {'Type': 'Water'}, 'pokemonsByType': 112},
 {'_id': {'Type': 'Ghost'}, 'pokemonsByType': 32},
 {'_id': {'Type': 'Ground'}, 'pokemonsByType': 32},
 {'_id': {'Type': 'Electric'}, 'pokemonsByType': 44},
 {'_id': {'Type': 'Ice'}, 'pokemonsByType': 24},
 {'_id': {'Type': 'Rock'}, 'pokemonsByType': 44},
 {'_id': {'Type': 'Fire'}, 'pokemonsByType': 52}]

#### Requête 5
Trouver les 3 types principaux (Type 1) ayant le plus de légendaires

In [112]:
cur = collection.aggregate([
    {"$match": {"Legendary": True}},
    {"$group" : {"_id" : {"Type": "$Type 1"}, "legendaryByType": {"$sum": 1}}},
    {'$sort': {"legendaryByType": -1}},
    {'$limit': 3}
])
list(cur)

[{'_id': {'Type': 'Psychic'}, 'legendaryByType': 14},
 {'_id': {'Type': 'Dragon'}, 'legendaryByType': 12},
 {'_id': {'Type': 'Fire'}, 'legendaryByType': 5}]

#### Requête 5
Compter le nombre de pokémons pour chaque type principal (Type 1) avec les caractéristiques suivantes:
- Leur nom qui commence par la lettre "G"
- Sont de génération 3 ou 4

On affichera:
- Le nombre total de pokémons
- Le nombre de pokémons légendaires
- Le nombre de pokémons non légendaires

On les triera par nombre total de pokémons

In [224]:
cur = collection.aggregate([
    {"$match": {
        "Name": {"$regex": '^G'},
        "$or" : [{"Generation": 3}, {"Generation": 4}],
    }},
    {"$group" : {
        "_id" : {"Type": "$Type 1"},
        "totalOfPokemons": {"$sum": 1},
        "legendaries": {"$sum": {"$cond": [{'$eq': [ '$Legendary', True ]}, 1, 0]}}},
    },
    {"$addFields": {
        "pokemons": {"$subtract": ["$totalOfPokemons", "$legendaries"]}
    }},
    {"$sort": {
        "totalOfPokemons": -1,
    }}
])
list(cur)

[{'_id': {'Type': 'Psychic'},
  'totalOfPokemons': 5,
  'legendaries': 0,
  'pokemons': 5},
 {'_id': {'Type': 'Dragon'},
  'totalOfPokemons': 4,
  'legendaries': 0,
  'pokemons': 4},
 {'_id': {'Type': 'Ground'},
  'totalOfPokemons': 3,
  'legendaries': 2,
  'pokemons': 1},
 {'_id': {'Type': 'Ice'},
  'totalOfPokemons': 3,
  'legendaries': 0,
  'pokemons': 3},
 {'_id': {'Type': 'Water'},
  'totalOfPokemons': 2,
  'legendaries': 0,
  'pokemons': 2},
 {'_id': {'Type': 'Ghost'},
  'totalOfPokemons': 2,
  'legendaries': 2,
  'pokemons': 0},
 {'_id': {'Type': 'Grass'},
  'totalOfPokemons': 2,
  'legendaries': 0,
  'pokemons': 2},
 {'_id': {'Type': 'Poison'},
  'totalOfPokemons': 1,
  'legendaries': 0,
  'pokemons': 1},
 {'_id': {'Type': 'Normal'},
  'totalOfPokemons': 1,
  'legendaries': 0,
  'pokemons': 1}]

#### Requête 6
Afficher la moyenne de points de vie, d'attaque, et de défense légendaires
On prendra les 3 types qui ont la meilleure moyenne d'attaque

In [241]:
cur = collection.aggregate([
    {"$match": {
        "Legendary": True
    }},
    {"$group" : {
        "_id" : {"Type": "$Type 1"},
        "averageHealth": {"$avg": "$HP"},
        "averageAttack": {"$avg": "$Attack"},
        "averageDefense": {"$avg": "$Defense"},
    }},
    {"$sort": {
        "averageAttack": -1
    }},
    {"$limit": 3}
])
list(cur)

[{'_id': {'Type': 'Ground'},
  'averageHealth': 94.5,
  'averageAttack': 150.0,
  'averageDefense': 120.0},
 {'_id': {'Type': 'Normal'},
  'averageHealth': 115.0,
  'averageAttack': 140.0,
  'averageDefense': 115.0},
 {'_id': {'Type': 'Fairy'},
  'averageHealth': 126.0,
  'averageAttack': 131.0,
  'averageDefense': 95.0}]